In [2]:
!pip install geemap


In [19]:
pip install earthengine-api


In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VMAaF7U1CV146E0c4Uii0Vq4t0fapxeAA2Kscec4WYgSGd-A74dc-c

Successfully saved authorization token.


In [2]:
ee.Initialize()

# Parameters


In [55]:
file_name = "isb_landsat"     #name of the file export (landSat Image)
pak_district = "Islamabad"    #name of district (for which data is required)
date_start = "2020-01-01"      #start date for data acquisition
date_end = "2021-01-01"        #last date for data acquisition

In [60]:
Map=geemap.Map()
country=ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
pak=ee.FeatureCollection("projects/ee-cep2/assets/Pak_adm3");

roi=pak.filter(ee.Filter.eq("NAME_3",pak_district));
Map.addLayer(roi,{},file_name)
Map.centerObject(roi,11);

#getting image collection
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1")\
.filterDate(date_start, date_end)\
.filterBounds(roi)\
.sort('CLOUD_COVER')


composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection':landsat,
    'asFloat': True
})
#Clipping img on the ROI
clipped_com = composite.clip(roi)

#setting bands for composite visuallization
rgbvis={'bands':["B4","B3","B2"], 'min':0, 'max':0.3}

#adding LandSat Image as layer
Map.addLayer(clipped_com, rgbvis, "Landsat Img")
Map



In [57]:
from pprint import pprint
pprint(clipped_com.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'B1'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'B2'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'B3'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'B4'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'B5'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 

In [58]:
#exporting image
task = ee.batch.Export.image.toDrive(
    image=clipped_com,
    description=file_name,
    folder='GISdata_cep',    #chamge folder as required
    region=roi.geometry(),
    scale=30
)
task.start()



In [59]:
#run to check status of Export to Drive
import time
print('Exporting image to Google Drive...')
while task.status()['state'] == 'RUNNING':
    print('Exporting...')
    time.sleep(20)
print('Export completed.')

Exporting image to Google Drive...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed.
